## Imports

In [1]:
from typing import Callable, Any, Dict
import re

from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing

from natsort import natsorted

## Global information about each suite

In [2]:
runs = {
    run: {"version": version.name}
    for version in Path("./out").iterdir()
    for run in version.iterdir()
}


def add_entry_to_run(
    new_key: str, f: Callable[[Path, Dict[str, Any]], Any]
) -> Dict[Path, Dict[str, Any]]:
    global runs
    for key, value in runs.items():
        value.update({new_key: f(key, value)})


add_entry_to_run("name", lambda p, d: p.name)

### Time

In [3]:
def get_time(p:Path, d):
    file_path = p.rglob("klee/info").__next__()
    if file_path.exists():
        with open(file_path, "r") as file:
            res = re.search(r"--max-time=(\w*)", file.read())
            if res:
                return res.group(1)

    print(f"Error for {p}")
    return None

add_entry_to_run("time", get_time)

## Per util information

### Setup

In [4]:
add_entry_to_run("df", lambda p, d: pd.DataFrame(columns=[e.name for e in p.iterdir() if e.is_dir()]))

def add_entry_for_utils(key: str, f: Callable[[Path], Any]) -> None:
    """
    Add entry for all utils

    Paramenters:
    key (str): key to add the new value at in the dataframe
    f (Callable[[Path], Any]): function taking the path to the subfolder for the util and returning the appropriate value
    """

    def adder(p: Path, d):
        df = d["df"]
        res = {}
        for util in df.columns:
            path = p / util
            if not path.exists():
                raise Exception(f"Path \"{util}\" does not exist")
            res[util] = f(path)
        df.loc[key] = res
        return df
    add_entry_to_run("df", adder)

### Number of errors according to KLEE

In [5]:
def read_num_errors(error_type:str) -> Callable[[Path], str]:
    def f(util_path: Path) -> str:
        file_path = util_path / "klee"
        if file_path.exists():
            return str(len(list(file_path.glob(f"*{error_type}.err"))))
        else:
            print(f"Error for {util_path}")
            return None
    return f
error_types = [file.name for run in runs.keys() for file in list(run.glob("**/*.err"))]
error_types = [e.split(".")[-2] for e in error_types]
error_types = list(set(error_types))
for error_type in error_types:
    add_entry_for_utils(f"num_errors ({error_type})", read_num_errors(error_type))
add_entry_for_utils(f"num_errors (total)", read_num_errors(""))

Error for out/coreutils-8.25/1h-3/hostname
Error for out/coreutils-8.25/1h-3/setuidgid
Error for out/coreutils-8.25/1h-2/hostname
Error for out/coreutils-8.25/1h-2/setuidgid
Error for out/coreutils-8.25/1h/hostname
Error for out/coreutils-8.25/1h/setuidgid
Error for out/coreutils-9.4/1h-3/hostname
Error for out/coreutils-9.4/1h-3/setuidgid
Error for out/coreutils-9.4/1h-2/hostname
Error for out/coreutils-9.4/1h-2/setuidgid
Error for out/coreutils-9.4/1h/hostname
Error for out/coreutils-9.4/1h/setuidgid
Error for out/coreutils-8.25/1h-3/hostname
Error for out/coreutils-8.25/1h-3/setuidgid
Error for out/coreutils-8.25/1h-2/hostname
Error for out/coreutils-8.25/1h-2/setuidgid
Error for out/coreutils-8.25/1h/hostname
Error for out/coreutils-8.25/1h/setuidgid
Error for out/coreutils-9.4/1h-3/hostname
Error for out/coreutils-9.4/1h-3/setuidgid
Error for out/coreutils-9.4/1h-2/hostname
Error for out/coreutils-9.4/1h-2/setuidgid
Error for out/coreutils-9.4/1h/hostname
Error for out/coreutils-9

### Coverage according to KLEE

In [6]:
def read_klee_csv(csv_name: str) -> Callable[[Path], str]:
    def f(util_path: Path) -> str:
        file_path = util_path / "klee-stats.csv"
        if file_path.exists() and file_path.stat().st_size > 0:
            df = pd.read_csv(file_path)
            return str(df[csv_name][0])
        else:
            print(f"Error for {csv_name} — {util_path}")
            return None
    return f

add_entry_for_utils("klee_ICov", read_klee_csv("ICov(%)"))
add_entry_for_utils("klee_BCov", read_klee_csv("BCov(%)"))

Error for ICov(%) — out/coreutils-8.25/1h-3/hostname
Error for ICov(%) — out/coreutils-8.25/1h-3/setuidgid
Error for ICov(%) — out/coreutils-8.25/1h-2/hostname
Error for ICov(%) — out/coreutils-8.25/1h-2/setuidgid
Error for ICov(%) — out/coreutils-8.25/1h/hostname
Error for ICov(%) — out/coreutils-8.25/1h/setuidgid
Error for ICov(%) — out/coreutils-9.4/1h-3/hostname
Error for ICov(%) — out/coreutils-9.4/1h-3/setuidgid
Error for ICov(%) — out/coreutils-9.4/1h-2/hostname
Error for ICov(%) — out/coreutils-9.4/1h-2/setuidgid
Error for ICov(%) — out/coreutils-9.4/1h/hostname
Error for ICov(%) — out/coreutils-9.4/1h/setuidgid
Error for BCov(%) — out/coreutils-8.25/1h-3/hostname
Error for BCov(%) — out/coreutils-8.25/1h-3/setuidgid
Error for BCov(%) — out/coreutils-8.25/1h-2/hostname
Error for BCov(%) — out/coreutils-8.25/1h-2/setuidgid
Error for BCov(%) — out/coreutils-8.25/1h/hostname
Error for BCov(%) — out/coreutils-8.25/1h/setuidgid
Error for BCov(%) — out/coreutils-9.4/1h-3/hostname
Err

### Coverage according to `gcov`

In [7]:
def read_gcov_cov(util_path: Path) -> str:
    file_path = util_path / "cov.txt"
    if file_path.exists():
        with open(file_path, "r") as file:
            res = re.search(r"File '(\.\./)?\.\./src/(\w+)\.c'\nLines executed:(\d?\d\d.\d\d)% of \d+", file.read())
            if res:
                return res.group(3)
    print(f"Error for {util_path}")
    return None

add_entry_for_utils("gcov_cov", read_gcov_cov)

Error for out/coreutils-8.25/1h-3/hostname
Error for out/coreutils-8.25/1h-3/[
Error for out/coreutils-8.25/1h-3/base64
Error for out/coreutils-8.25/1h-3/setuidgid
Error for out/coreutils-8.25/1h-3/md5sum
Error for out/coreutils-8.25/1h-3/ginstall
Error for out/coreutils-8.25/1h-2/hostname
Error for out/coreutils-8.25/1h-2/[
Error for out/coreutils-8.25/1h-2/base64
Error for out/coreutils-8.25/1h-2/setuidgid
Error for out/coreutils-8.25/1h-2/md5sum
Error for out/coreutils-8.25/1h-2/ginstall
Error for out/coreutils-8.25/1h/hostname
Error for out/coreutils-8.25/1h/[
Error for out/coreutils-8.25/1h/base64
Error for out/coreutils-8.25/1h/setuidgid
Error for out/coreutils-8.25/1h/md5sum
Error for out/coreutils-8.25/1h/ginstall
Error for out/coreutils-6.10/24h-3/md5sum
Error for out/coreutils-6.10/24h-3/ginstall
Error for out/coreutils-6.10/6h-3/uniq
Error for out/coreutils-6.10/6h-3/who
Error for out/coreutils-6.10/6h-3/md5sum
Error for out/coreutils-6.10/6h-3/tsort
Error for out/coreutils-

## Plots
### Massaging `df`s together

In [8]:
dfs = []
for k, v in runs.items():
    df = v["df"]
    df = df.reset_index(names="key")
    df = df.melt(id_vars="key", var_name="util")
    df["run"] = k.name
    df["time"] = v["time"]
    df["version"] = v["version"]
    dfs.append(df)

combined_df = pd.concat(dfs)
combined_df['value'] = combined_df['value'].astype(np.float64)
combined_df = combined_df.dropna(subset=['value'])
combined_df = combined_df.reset_index(drop=True)
keys_nat = {
    "gcov_cov": "coverage: line (gcov) (%)",
    "klee_ICov": "coverage: instruction (KLEE) (%)",
    "klee_BCov": "coverage: branch (KLEE) (%)",
    "num_errors (abort)": "error count: abort",
    "num_errors (exec)": "error count: exec",
    "num_errors (model)": "error count: model",
    "num_errors (ptr)": "error count: pointer",
    "num_errors (solver)": "error count: solver",
    "num_errors (total)": "error count: total",
}

path_by_key = {v:k for k,v in keys_nat.items()}

combined_df = combined_df.copy(deep=True)
combined_df["key"] = combined_df["key"].map(keys_nat)
print(combined_df.sample(20))

dpi = 100


                                    key    util  value      run     time  \
5026        coverage: branch (KLEE) (%)      id  29.62     6h-2   360min   
2994               error count: pointer   chgrp   0.00    24h-3  1440min   
11166               error count: solver    sort   0.00       1h    60min   
833                 error count: solver  unlink   0.00     1h-2    60min   
13967               error count: solver  printf   0.00     1h-2    60min   
6168                error count: solver     tee   0.00     1h-3    60min   
10532                error count: abort  mkfifo   0.00  10min-3    10min   
2318          coverage: line (gcov) (%)    expr  89.91       1h    60min   
3421   coverage: instruction (KLEE) (%)  mkfifo  41.42     6h-3   360min   
10716  coverage: instruction (KLEE) (%)   cksum  42.54  10min-3    10min   
12541                error count: abort   false   0.00  10min-2    10min   
4921               error count: pointer    stat   0.00     6h-2   360min   
13787  cover

### Plots by coverage

In [9]:
def paint_util(args):
    key, key_df = args

    path = f"plots/{path_by_key[key]}"
    Path(path).mkdir(exist_ok=True, parents=True)
    # By version
    fixed_time_df = key_df[key_df["time"] == "60min"].drop(columns="time")
    versions = natsorted(fixed_time_df["version"].unique())
    fig, axes = plt.subplots(
        ncols=1,
        nrows=len(versions),
        figsize=(6, 3 * len(versions)),
        dpi=dpi,
    )
    for version_i, version in enumerate(versions):
        version_df = fixed_time_df[fixed_time_df["version"] == version].drop(
            columns="version"
        )
        ax = axes[version_i]
        ax.set_title(version)
        for run in version_df["run"].unique():
            run_df = version_df[version_df["run"] == run].drop(columns="run")
            sns.ecdfplot(
                y="value",
                data=run_df,
                ax=ax,
                stat="count",
            )
        ax.set_ylim(ymin=0)
        ax.set_ylabel(key)
        ax.set_xlabel(None)
    fig.tight_layout()
    fig.savefig(path + "/ecdf_by_version.png")
    plt.close(fig)

    # By time
    fixed_version_df = key_df[key_df["version"] == "coreutils-6.10"].drop(columns="version")
    times = natsorted(fixed_version_df["time"].unique())
    fig, axes = plt.subplots(
        ncols=2,
        nrows=len(times) // 2,
        figsize=(6 * 2, 3 * len(times) // 2),
        dpi=dpi,
    )

    axes = axes.flatten()
    for time_i, time in enumerate(times):
        time_df = fixed_version_df[fixed_version_df["time"] == time].drop(
            columns="time"
        )

        ax = axes[time_i]
        ax.set_title(time)
        for run in time_df["run"].unique():
            run_df = time_df[time_df["run"] == run].drop(columns="run")
            sns.ecdfplot(
                y="value",
                data=run_df,
                ax=ax,
                stat="count",
            )
        ax.set_ylim(ymin=0)
        ax.set_ylabel(key)
        ax.set_xlabel(None)
    fig.tight_layout()
    fig.savefig(path + "/ecdf_by_time.png")
    plt.close(fig)


with multiprocessing.Pool() as pool:
    df = combined_df.copy(deep=True)
    pool.map(
        paint_util,
        [
            (key, df[df["key"] == key].drop(columns="key"))
            for key in natsorted(df["key"].unique())
        ],
    )

### Gains by time

In [10]:
df = combined_df.copy(deep=True)
df = df[df["version"] == "coreutils-6.10"].drop(columns=["run", "version"])
df = df.groupby(["key", "time", "util"], as_index=False).mean()
keys = natsorted(df["key"].unique())
fig, axes = plt.subplots(nrows=len(keys), ncols=1, figsize=(5, 3 * len(keys)), dpi=dpi)
for i, key in enumerate(keys):
    df_keys = df[df["key"] == key]
    df_keys = df_keys.drop(columns="key")
    order = natsorted(df_keys["time"].unique())
    df_keys["time"] = pd.Categorical(df_keys["time"], categories=order, ordered=True)
    df_keys = df_keys.sort_values(["util", "time"])
    df_keys["difference"] = df_keys.groupby("util", as_index=False)["value"].diff()
    df_keys = df_keys.drop(columns=["util", "value"])
    df_keys = df_keys.groupby(["time"], as_index=False, observed=True).mean()
    df_keys["time"] = (
        df_keys["time"].shift(1).astype(str) + " - " + df_keys["time"].astype(str)
    )
    df_keys = df_keys.dropna()
    sns.barplot(data=df_keys, x="time", y="difference", ax=axes[i])
    axes[i].set_ylabel(f"average {key} gained")
plt.tight_layout()
plt.savefig(f"plots/gains_by_time.png")
plt.close()

### Changes

### Plots by util

In [11]:
args_stripplot = {
    # "alpha": 0.5,
    "size": 2,
    "y": "value_norm",
    "jitter": 0.3,
}
args_pointplot = {
    "y": "value_norm",
    "color": "red",
    "markers": "_",
    "markersize": 100,
    "markeredgewidth": 1,
    "linestyles": "none",
    "zorder": 10,
}

handles = [
    plt.Line2D(
        [], [], linestyle="none", marker="o", markersize=2, color="blue", label="values"
    ),
    plt.Line2D([], [], color="red", label="average"),
]


def paint_util(args):
    key, df = args
    df_norm = df[(df["time"] == "60min") & (df["version"] == "coreutils-6.10")]
    df_norm = df_norm.drop(columns=["time", "version", "run"])
    df_norm = df_norm.groupby(by="util", as_index=False, observed=True)["value"].mean()
    df_norm = df_norm.rename(columns={"value": "value_avg"})
    df = df.merge(df_norm, on="util", how="left")
    df["value_norm"] = df["value"] - df["value_avg"]
    df_version = df[df["version"] == "coreutils-6.10"]
    df_time = df[df["time"] == "60min"]
    times = natsorted(df["time"].unique())
    versions = natsorted(df["version"].unique())
    fig, ax = plt.subplots(1, 1, figsize=(8, 4), dpi=dpi)

    def get_avg(df_to_average: pd.DataFrame, key: str) -> pd.DataFrame:
        return (
            df_to_average[[key, "value_norm"]]
            .groupby(key, as_index=False, observed=True)["value_norm"]
            .mean()
        )

    sns.stripplot(data=df_version, ax=ax, x="time", order=times, **args_stripplot)
    sns.pointplot(
        data=get_avg(df_version, "time"),
        ax=ax,
        x="time",
        order=times,
        **args_pointplot,
    )

    ax.set_ylabel(u'Δ '+key)
    ax.legend(handles=handles)
    fig.tight_layout()
    path = f"plots/{path_by_key[key]}"
    Path(path).mkdir(exist_ok=True, parents=True)
    fig.savefig(path+"/changes-by-time.png")
    plt.close(fig)

    fig, ax = plt.subplots(1, 1, figsize=(8, 4), dpi=dpi)

    sns.stripplot(data=df_time, ax=ax, x="version", order=versions, **args_stripplot)
    sns.pointplot(
        data=get_avg(df_time, "version"),
        ax=ax,
        x="version",
        order=versions,
        **args_pointplot,
    )
    ax.set_ylabel(u'Δ '+key)
    ax.legend(handles=handles)

    fig.tight_layout()
    fig.savefig(path+"/changes-by-version.png")
    plt.close(fig)


with multiprocessing.Pool() as pool:
    df = combined_df.copy(deep=True)
    pool.map(
        paint_util,
        [
            (key, df[df["key"] == key].drop(columns="key"))
            for key in natsorted(df["key"].unique())
        ],
    )

In [12]:
def paint_util(args):
    key, util, key_df = args
    util_df = key_df[key_df["util"] == util].drop(columns="util")
    util_df = util_df.sort_values(by="version")
    times = natsorted(util_df['time'].unique())
    fig, ax = plt.subplots(1, 1, figsize=(8, 6), dpi=dpi)
    sns.stripplot(data=util_df, ax=ax, x='time', y="value", hue="version", order=times)
    ax.set_title(f"{util}")
    ax.set_ylabel(key)
    fig.tight_layout()
    path = f"plots/{path_by_key[key]}/by-util"
    Path(path).mkdir(exist_ok=True, parents=True)
    fig.savefig(path + f"/{util}.png")
    plt.close(fig)

df = combined_df.copy(deep=True)
for key in natsorted(df["key"].unique()):
    key_df = df[df["key"] == key].drop(columns="key")
    
    utils = natsorted(key_df["util"].unique())

    with multiprocessing.Pool() as pool:
        pool.map(paint_util, [(key, util, key_df) for util in utils])
    
    print(f"Done with {key}")


Done with coverage: branch (KLEE) (%)
Done with coverage: instruction (KLEE) (%)
Done with coverage: line (gcov) (%)
Done with error count: abort
Done with error count: exec
Done with error count: model
Done with error count: pointer
Done with error count: solver
Done with error count: total


## Plots for the Report

### coreutils 6.10 @60min

In [13]:
df = combined_df.copy(deep=True)

c6_60_df = df[(df["version"] == "coreutils-6.10") & (df["time"] == "60min")].drop(
    columns=["version", "time"]
)

#### Spreads

In [14]:
df = c6_60_df.copy(deep=True)
averages = df.groupby(by=["util", "key"], as_index=False, observed=True)
averages = averages["value"].mean()
averages = averages.rename(columns={"value": "value_avg"})
df = df.merge(averages, on=["util", "key"], how="left")
df["value_norm"] = df["value"] - df["value_avg"]
keys = natsorted(df["key"].unique())
keys = [key for key in keys if "total" not in key]
ncols = np.ceil(len(keys) / 2).astype(int)
fig, ax = plt.subplots(2, ncols, figsize=(3 * ncols, 6 * 2), dpi=dpi)
ax = ax.flatten()
for i, key in enumerate(keys):
    key_df = df[df["key"] == key].drop(columns="key")
    sns.stripplot(data=key_df, ax=ax[i], **args_stripplot)
    print(key, np.round(key_df["value_norm"].std(),2))
    ax[i].set_ylabel(None)
    ax[i].set_xlabel(u'Δ '+key)
plt.tight_layout()
fig.savefig("./report/assets/spread_60min_6.10.png")
plt.close(fig)

coverage: branch (KLEE) (%) 0.37
coverage: instruction (KLEE) (%) 0.48
coverage: line (gcov) (%) 1.56
error count: abort 0.0
error count: exec 0.0
error count: model 0.1
error count: pointer 0.33
error count: solver 0.11


In [15]:
df = c6_60_df.copy(deep=True)
df = df[df['key'] == 'coverage: line (gcov) (%)']
fig, ax = plt.subplots(1, 1, figsize=(4.5, 3), dpi=2*dpi)
for run in df["run"].unique():
    sns.ecdfplot(
        y="value",
        data=df[df["run"] == run],
        ax=ax,
        stat="count",
    )
ax.set_ylim(ymin=0)
ax.set_ylabel('coverage: line (gcov) (%)')
ax.set_xlabel(None)
fig.tight_layout()
fig.savefig('./report/assets/ecdf_60min_6.10.png')
plt.close(fig)